In [797]:
#import nessecary package
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import requests
import lxml.html as lh
import geocoder
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [798]:
#import Boston Neighbourhood Data
boston = pd.read_csv(r'C:\Users\watch\Desktop\projects\Coursera_Capstone\boston.csv',index_col=0)

In [799]:
boston.head()

,Neighbourhood,Latitude,Longitude,Population Density
0,Roslindale,42.291209,-71.124497,9877.31
1,Jamaica Plain,42.309820,-71.120330,10363.86
2,Mission Hill,42.332926,-71.103214,22637.14
3,Longwood,42.336168,-71.099527,17561.48
4,Bay Village,42.350011,-71.066948,49840.22


In [800]:
#inspect data sorted by Population Density
boston.loc[:,['Neighbourhood','Population Density']].sort_values(by='Population Density',ascending=False)

,Neighbourhood,Population Density
5,Chinatown,52322.61
4,Bay Village,49840.22
13,Beacon Hill,47770.65
6,North End,42239.76
15,Fenway,37978.28
8,South End,32320.73
9,Back Bay,26282.87
2,Mission Hill,22637.14
23,Allston,19029.08
3,Longwood,17561.48


In [801]:
#drop south boston waterfront because it's the same lat and lng as south boston which will results in duplicate data from foursquare api later on
boston.drop(21,axis=0,inplace=True)
boston

,Neighbourhood,Latitude,Longitude,Population Density
0,Roslindale,42.291209,-71.124497,9877.31
1,Jamaica Plain,42.309820,-71.120330,10363.86
2,Mission Hill,42.332926,-71.103214,22637.14
3,Longwood,42.336168,-71.099527,17561.48
4,Bay Village,42.350011,-71.066948,49840.22
5,Chinatown,42.352217,-71.062607,52322.61
6,North End,42.365097,-71.054495,42239.76
7,Roxbury,42.324843,-71.095016,14810.95
8,South End,42.341310,-71.077230,32320.73
9,Back Bay,42.350707,-71.079730,26282.87


In [802]:
#inspect stat from Population Density 
boston.iloc[:,-1:].describe()

,Population Density
count,24.000000
mean,20762.952917
std,15015.374429
min,1571.750000
25%,10242.222500
50%,15730.400000
75%,27792.335000
max,52322.610000


In [803]:
#retrieve boston lat and lng
address = 'Boston'

geolocator = Nominatim(user_agent="boston_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


In [804]:
# create map of Boston using latitude and longitude values
map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(boston['Latitude'], boston['Longitude'],boston['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

In [805]:
CLIENT_ID = 'AXAN1LJCFHFD0V2TP14TXXZN4ABNLAOBAXH2QOPXUREMUA3D' # your Foursquare ID
CLIENT_SECRET = 'XHCUXSLLMYO0BG1TNVTU21IMJUYEHLHNCITKBIEWNK0VGZK4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AXAN1LJCFHFD0V2TP14TXXZN4ABNLAOBAXH2QOPXUREMUA3D
CLIENT_SECRET:XHCUXSLLMYO0BG1TNVTU21IMJUYEHLHNCITKBIEWNK0VGZK4


In [806]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,Limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for x in venues_list for item in x])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [807]:
#get the venue information from each neighbourhood
boston_venues = getNearbyVenues(names=boston['Neighbourhood'],
                                   latitudes=boston['Latitude'],
                                   longitudes=boston['Longitude']
                                  )

Roslindale
Jamaica Plain
Mission Hill
Longwood
Bay Village
Chinatown
North End
Roxbury
South End
Back Bay
East Boston
Charlestown
West End
Beacon Hill
Downtown
Fenway
Brighton
West Roxbury
Hyde Park
Mattapan
Dorchester
South Boston
Allston
Harbor Islands


In [808]:
print(boston_venues.shape)

(1096, 7)


In [809]:
print('There are {} uniques categories.'.format(len(boston_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [834]:
#inspect stat from foursquare data
boston_venues.loc[:,['Venue','Venue Category']].describe(include='all')

,Venue,Venue Category
count,1096,1096
unique,894,200
top,Dunkin' Donuts,Italian Restaurant
freq,15,74


In [811]:
boston_venues['Venue Category'].unique()[0:30]

array(['Scenic Lookout', 'Pizza Place', 'American Restaurant',
       'Big Box Store', 'Pool', 'Rental Car Location', 'Donut Shop',
       'Yoga Studio', 'Trail', 'Accessories Store', 'Bakery', 'Theater',
       'Pet Store', 'Liquor Store', 'Coffee Shop', 'Deli / Bodega',
       'Seafood Restaurant', 'Art Gallery', 'Library', 'Park', 'Bar',
       'Italian Restaurant', 'Gastropub', 'New American Restaurant',
       'Ice Cream Shop', 'Sandwich Place', 'Caribbean Restaurant', 'Pub',
       'Greek Restaurant', 'Falafel Restaurant'], dtype=object)

In [812]:
# one hot encoding
boston_onehot = pd.get_dummies(boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighbourhood'] = boston_venues['Neighbourhood']

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + boston_onehot.columns[:-1].tolist()
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head(10)

,Neighbourhood,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tourist Information Center,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Roslindale,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,Roslindale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [813]:
print(boston_onehot.shape)

(1096, 201)


In [814]:
#groupby neighbourhood and using mean function
boston_grouped = boston_onehot.groupby('Neighbourhood').mean().reset_index()
boston_grouped

,Neighbourhood,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Tourist Information Center,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Allston,0.000000,0.000000,0.000000,0.012346,0.000000,0.000000,0.024691,0.000000,0.000000,...,0.00,0.000000,0.000000,0.024691,0.00,0.012346,0.000000,0.000000,0.00,0.000000
1,Back Bay,0.020000,0.050000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.000000,0.02,0.010000
2,Bay Village,0.000000,0.031250,0.000000,0.000000,0.000000,0.010417,0.020833,0.000000,0.000000,...,0.00,0.000000,0.000000,0.020833,0.00,0.020833,0.000000,0.010417,0.00,0.010417
3,Beacon Hill,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.016129
4,Brighton,0.000000,0.020833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020833,0.00,0.000000
5,Charlestown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000
6,Chinatown,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.000000,...,0.01,0.000000,0.000000,0.020000,0.00,0.020000,0.010000,0.000000,0.00,0.000000
7,Dorchester,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.058824,0.00,0.000000,0.000000,0.000000,0.00,0.058824
8,Downtown,0.000000,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021739,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.021739
9,East Boston,0.000000,0.029412,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.029412,0.000000,0.000000,0.00,0.000000


In [815]:
#display top 3 venue from each neighbourhood
num_top_venues = 3

for hood in boston_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                venue  freq
0   Korean Restaurant  0.07
1  Chinese Restaurant  0.04
2         Pizza Place  0.04


----Back Bay----
                 venue  freq
0  American Restaurant  0.05
1          Coffee Shop  0.05
2   Italian Restaurant  0.04


----Bay Village----
                venue  freq
0               Hotel  0.05
1      Sandwich Place  0.05
2  Italian Restaurant  0.04


----Beacon Hill----
                venue  freq
0  Italian Restaurant  0.06
1         Pizza Place  0.05
2           Hotel Bar  0.05


----Brighton----
                venue  freq
0         Coffee Shop  0.06
1  Chinese Restaurant  0.06
2              Bakery  0.06


----Charlestown----
         venue  freq
0  Pizza Place  0.12
1  Coffee Shop  0.09
2    Gastropub  0.06


----Chinatown----
                venue  freq
0  Chinese Restaurant  0.11
1    Asian Restaurant  0.07
2              Bakery  0.05


----Dorchester----
            venue  freq
0     Yoga Studio  0.06
1  Breakfast Spot  0.06
2    Ho

In [816]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [817]:
num_top_venues = 10

In [818]:
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = boston_grouped['Neighbourhood']

In [819]:
#create dataframe to display top common venues
for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

#preparing data for model
boston_grouped = pd.merge(boston_grouped,boston.iloc[:,[0,3]],how='outer',on=['Neighbourhood','Neighbourhood'])
boston_grouped_clustering = boston_grouped.drop('Neighbourhood', 1)
#drop neighbourhood that we didn't obtain data
boston_grouped_clustering.dropna(axis=0,inplace=True)

In [820]:
# set number of clusters
kclusters = 4
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 1, 1, 0, 3, 1, 0, 3, 3])

In [821]:
# add clustering labels 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_ + 1)

In [822]:
boston_merged = boston

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')


In [823]:
boston_merged.dropna(axis=0,inplace=True)
boston_merged.reset_index(inplace=True)
boston_merged.head()

,index,Neighbourhood,Latitude,Longitude,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Roslindale,42.291209,-71.124497,9877.31,4.0,Yoga Studio,Pizza Place,American Restaurant,Scenic Lookout,Donut Shop,Pool,Big Box Store,Rental Car Location,Dive Bar,Falafel Restaurant
1,1,Jamaica Plain,42.309820,-71.120330,10363.86,4.0,Art Gallery,Bakery,Coffee Shop,Accessories Store,Deli / Bodega,Liquor Store,Library,Donut Shop,Pizza Place,Theater
2,2,Mission Hill,42.332926,-71.103214,22637.14,1.0,Sushi Restaurant,Sandwich Place,Pizza Place,Convenience Store,Greek Restaurant,Caribbean Restaurant,Gastropub,Café,Coffee Shop,Bar
3,3,Longwood,42.336168,-71.099527,17561.48,1.0,Donut Shop,Sandwich Place,Italian Restaurant,Gastropub,Coffee Shop,Metro Station,Falafel Restaurant,Café,Liquor Store,Bookstore
4,4,Bay Village,42.350011,-71.066948,49840.22,2.0,Sandwich Place,Hotel,Steakhouse,Performing Arts Venue,Theater,Italian Restaurant,Seafood Restaurant,Spa,Gym / Fitness Center,Coffee Shop


In [824]:
#change type of cluster from float to int for easy inspect
boston_merged['Cluster Labels'] = boston_merged['Cluster Labels'].astype(int)

In [825]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitude'], boston_merged['Longitude'], boston_merged['Neighbourhood'], boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1 ],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [826]:
boston_merged.loc[boston_merged['Cluster Labels'] == 1,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Mission Hill,22637.14,1,Sushi Restaurant,Sandwich Place,Pizza Place,Convenience Store,Greek Restaurant,Caribbean Restaurant,Gastropub,Café,Coffee Shop,Bar
3,Longwood,17561.48,1,Donut Shop,Sandwich Place,Italian Restaurant,Gastropub,Coffee Shop,Metro Station,Falafel Restaurant,Café,Liquor Store,Bookstore
7,Roxbury,14810.95,1,Art Gallery,Plaza,Furniture / Home Store,Gym,Metro Station,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant
12,West End,17420.70,1,Sandwich Place,Pizza Place,Donut Shop,Coffee Shop,Hotel,Bar,Italian Restaurant,Café,Mexican Restaurant,Sports Bar
16,Brighton,16433.47,1,Chinese Restaurant,Pizza Place,Pub,Coffee Shop,Bakery,Dry Cleaner,Bus Station,Grocery Store,Deli / Bodega,Bank
19,Dorchester,15027.33,1,Yoga Studio,Southern / Soul Food Restaurant,Gym,Fried Chicken Joint,Market,Mobile Phone Shop,Donut Shop,Diner,Nail Salon,Pizza Place
21,Allston,19029.08,1,Korean Restaurant,Pizza Place,Chinese Restaurant,Bakery,Thai Restaurant,Dive Bar,Donut Shop,Bubble Tea Shop,Italian Restaurant,Pharmacy


In [827]:
boston_merged.loc[boston_merged['Cluster Labels'] == 2,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Bay Village,49840.22,2,Sandwich Place,Hotel,Steakhouse,Performing Arts Venue,Theater,Italian Restaurant,Seafood Restaurant,Spa,Gym / Fitness Center,Coffee Shop
5,Chinatown,52322.61,2,Chinese Restaurant,Asian Restaurant,Bakery,Coffee Shop,Theater,Sushi Restaurant,Performing Arts Venue,Pizza Place,Sandwich Place,Mexican Restaurant
6,North End,42239.76,2,Italian Restaurant,Pizza Place,Park,Seafood Restaurant,Bakery,Wine Shop,Café,Coffee Shop,Market,Sandwich Place
13,Beacon Hill,47770.65,2,Italian Restaurant,Hotel Bar,Pizza Place,French Restaurant,Museum,Hotel,Gourmet Shop,Gift Shop,Sushi Restaurant,Plaza


In [828]:
boston_merged.loc[boston_merged['Cluster Labels'] == 3,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,South End,32320.73,3,Italian Restaurant,Coffee Shop,Gift Shop,Wine Shop,Wine Bar,Park,Bakery,Bar,Latin American Restaurant,Japanese Restaurant
9,Back Bay,26282.87,3,American Restaurant,Coffee Shop,Italian Restaurant,Seafood Restaurant,Hotel,Sporting Goods Shop,Clothing Store,Cosmetics Shop,Spa,Salon / Barbershop
15,Fenway,37978.28,3,Sports Bar,Pizza Place,Lounge,Coffee Shop,American Restaurant,Restaurant,Baseball Field,Thai Restaurant,Brewery,Mexican Restaurant


In [829]:
boston_merged.loc[boston_merged['Cluster Labels'] == 4,boston_merged.columns[[1,4,5,6,7,8,9,10,11,12,13,14,15]]]

,Neighbourhood,Population Density,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roslindale,9877.31,4,Yoga Studio,Pizza Place,American Restaurant,Scenic Lookout,Donut Shop,Pool,Big Box Store,Rental Car Location,Dive Bar,Falafel Restaurant
1,Jamaica Plain,10363.86,4,Art Gallery,Bakery,Coffee Shop,Accessories Store,Deli / Bodega,Liquor Store,Library,Donut Shop,Pizza Place,Theater
10,East Boston,8594.21,4,Latin American Restaurant,Sandwich Place,Convenience Store,Pizza Place,Donut Shop,Colombian Restaurant,Pharmacy,Mexican Restaurant,Restaurant,Fast Food Restaurant
11,Charlestown,12192.04,4,Pizza Place,Coffee Shop,Convenience Store,Pub,Liquor Store,Gastropub,Shopping Mall,Donut Shop,Discount Store,Café
14,Downtown,5975.80,4,Italian Restaurant,Hotel Bar,Gift Shop,Museum,Gourmet Shop,Baseball Field,Bistro,Playground,Pizza Place,Other Repair Shop
17,Hyde Park,7474.90,4,American Restaurant,Pizza Place,Bar,Gym,Donut Shop,Discount Store,Platform,Theater,Italian Restaurant,Fast Food Restaurant
18,Mattapan,12753.33,4,Home Service,Bakery,Park,Scenic Lookout,Liquor Store,Furniture / Home Store,Caribbean Restaurant,Rental Car Location,Ice Cream Shop,Cosmetics Shop
20,South Boston,12192.04,4,Chinese Restaurant,Convenience Store,Pizza Place,Italian Restaurant,Cosmetics Shop,Bar,Liquor Store,Sports Bar,Coffee Shop,Ice Cream Shop
